In [ ]:
import nbtest
import json
import numpy as np
random_seed = np.random.randint(10000)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('./heart_failure.csv')
nbtest.assert_df_var(df, 5655280085.135918, atol=0.0, test_id='4')
nbtest.assert_df_mean(df, 20331.504464625676, atol=0.0, test_id='3')
nbtest.assert_column_types(df, ['int64', 'float64', 'int64', 'int64', 'int64', 'int64', 'int64', 'float64', 'float64', 'int64', 'int64', 'int64', 'int64'], test_id='2')
nbtest.assert_column_names(df, ['DEATH_EVENT', 'age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time'], test_id='1')
nbtest.assert_shape(df, (299, 12), test_id='0')

In [ ]:
nbtest_tmpvar_0 = df.head(n=10)
nbtest.assert_df_var(nbtest_tmpvar_0, 5735375787.815456, atol=0.0, test_id='9')
nbtest.assert_df_mean(nbtest_tmpvar_0, 20584.102, atol=0.0, test_id='8')
nbtest.assert_column_types(nbtest_tmpvar_0, ['int64', 'float64', 'int64', 'int64', 'int64', 'int64', 'int64', 'float64', 'float64', 'int64', 'int64', 'int64', 'int64'], test_id='7')
nbtest.assert_column_names(nbtest_tmpvar_0, ['DEATH_EVENT', 'age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time'], test_id='6')
nbtest.assert_shape(nbtest_tmpvar_0, (10, 13), test_id='5')

In [ ]:
df.info()

In [ ]:
nbtest_tmpvar_2 = df.isnull().any()
nbtest.assert_allclose(nbtest_tmpvar_2.sum(), 0.0, atol=0.0, test_id='10')

In [ ]:
df.shape
nbtest.assert_shape(df, (299, 13), test_id='11')

In [ ]:
nbtest_tmpvar_3 = df.describe()
nbtest.assert_df_var(nbtest_tmpvar_3, 9311258132.272377, atol=0.0, test_id='16')
nbtest.assert_df_mean(nbtest_tmpvar_3, 19528.024571370242, atol=0.0, test_id='15')
nbtest.assert_column_types(nbtest_tmpvar_3, ['float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64'], test_id='14')
nbtest.assert_column_names(nbtest_tmpvar_3, ['DEATH_EVENT', 'age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time'], test_id='13')
nbtest.assert_shape(nbtest_tmpvar_3, (8, 13), test_id='12')

In [ ]:
for i in df.columns:
  print(i)
  print(df[f"{i}"].value_counts())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.pairplot(df, vars=['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time','DEATH_EVENT'], hue='DEATH_EVENT')
plt.show()

In [ ]:
korelasyon_matrisi = df.corr().abs()
plt.figure(figsize=(8, 6), dpi=100)
sns.heatmap(korelasyon_matrisi, annot=True, cmap='rocket_r', fmt=".2f", linewidths=1)
plt.title("Korelasyon Heatmap")
plt.show()

In [ ]:
print(df.corr().abs()["DEATH_EVENT"].nlargest(15))

In [ ]:
col_reduced = df.corr().abs()["DEATH_EVENT"].nlargest(6).index
col_reduced

In [ ]:
x = df[['time', 'serum_creatinine', 'ejection_fraction', 'age', 'serum_sodium']].values # Numpy Array
y = df.iloc[:,-1].values # Numpy Array

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [ ]:
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
logr = LogisticRegression(random_state=0)
nbtest_tmpvar_4 = logr.fit(X_train, y_train)
nbtest.assert_sklearn_model(nbtest_tmpvar_4, '/home/verve/ml_nb_testing/comp_analysis/nbtest-gen/example/op/sklearn_model_17.pkl', test_id='17')
y_pred = logr.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=logr.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=logr.classes_)
disp.plot(cmap='YlGnBu')
plt.show()
print()
print('accuracy: ', accuracy_score(y_test, y_pred))
nbtest.assert_allclose(accuracy_score(y_test, y_pred), 0.8400000000000001, atol=0.11155467020454334, test_id='18')

In [ ]:
import sklearn
y_proba = logr.predict_proba(X_test)
(fpr, tpr, thresholds) = roc_curve(y_test, y_proba[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)
nbtest.assert_allclose(roc_auc, 0.9108934567101145, atol=0.1456452575839852, test_id='19')
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()
print(sklearn.metrics.roc_auc_score(y_test, y_pred))
nbtest.assert_allclose(sklearn.metrics.roc_auc_score(y_test, y_pred), 0.8110204833719143, atol=0.14027071488727222, test_id='20')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
nbtest_tmpvar_5 = knn.fit(X_train, y_train)
nbtest.assert_sklearn_model(nbtest_tmpvar_5, '/home/verve/ml_nb_testing/comp_analysis/nbtest-gen/example/op/sklearn_model_21.pkl', test_id='21')
y_pred = knn.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=knn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=knn.classes_)
disp.plot(cmap='YlGnBu')
plt.show()
print()
print('accuracy: ', accuracy_score(y_test, y_pred))
nbtest.assert_allclose(accuracy_score(y_test, y_pred), 0.8, atol=0.1563471919941143, test_id='22')

In [ ]:
import matplotlib.pyplot as plt
y_proba = knn.predict_proba(X_test)
(fpr, tpr, thresholds) = roc_curve(y_test, y_proba[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)
nbtest.assert_allclose(roc_auc, 0.852153813699536, atol=0.20428763685163875, test_id='23')
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()
print(sklearn.metrics.roc_auc_score(y_test, y_pred))
nbtest.assert_allclose(sklearn.metrics.roc_auc_score(y_test, y_pred), 0.7645997097367632, atol=0.1841917680044929, test_id='24')

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='rbf', probability=True)
nbtest_tmpvar_6 = svc.fit(X_train, y_train)
nbtest.assert_sklearn_model(nbtest_tmpvar_6, '/home/verve/ml_nb_testing/comp_analysis/nbtest-gen/example/op/sklearn_model_25.pkl', test_id='25')
y_pred = svc.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=svc.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=svc.classes_)
disp.plot(cmap='YlGnBu')
plt.show()
print()
print('accuracy: ', accuracy_score(y_test, y_pred))
nbtest.assert_allclose(accuracy_score(y_test, y_pred), 0.7933333333333333, atol=0.15347819244295127, test_id='26')

In [ ]:
import matplotlib.pyplot as plt
y_proba = svc.predict_proba(X_test)
(fpr, tpr, thresholds) = roc_curve(y_test, y_proba[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)
nbtest.assert_allclose(roc_auc, 0.8907349966966368, atol=0.12229281055777137, test_id='27')
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()
print(sklearn.metrics.roc_auc_score(y_test, y_pred))
nbtest.assert_allclose(sklearn.metrics.roc_auc_score(y_test, y_pred), 0.7559464317638447, atol=0.23756857957372185, test_id='28')

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
nbtest_tmpvar_7 = gnb.fit(X_train, y_train)
nbtest.assert_sklearn_model(nbtest_tmpvar_7, '/home/verve/ml_nb_testing/comp_analysis/nbtest-gen/example/op/sklearn_model_29.pkl', test_id='29')
y_pred = gnb.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=gnb.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gnb.classes_)
disp.plot(cmap='YlGnBu')
plt.show()
print()
print('accuracy: ', accuracy_score(y_test, y_pred))
nbtest.assert_allclose(accuracy_score(y_test, y_pred), 0.8266666666666665, atol=0.12110601416389959, test_id='30')

In [ ]:
import matplotlib.pyplot as plt
y_proba = gnb.predict_proba(X_test)
(fpr, tpr, thresholds) = roc_curve(y_test, y_proba[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)
nbtest.assert_allclose(roc_auc, 0.9026079458477707, atol=0.16769145439539307, test_id='31')
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()
print(sklearn.metrics.roc_auc_score(y_test, y_pred))
nbtest.assert_allclose(sklearn.metrics.roc_auc_score(y_test, y_pred), 0.7738830023961791, atol=0.25162112938452746, test_id='32')

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion='entropy', random_state=10)
nbtest_tmpvar_8 = dtc.fit(X_train, y_train)
nbtest.assert_sklearn_model(nbtest_tmpvar_8, '/home/verve/ml_nb_testing/comp_analysis/nbtest-gen/example/op/sklearn_model_33.pkl', test_id='33')
y_pred = dtc.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=dtc.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dtc.classes_)
disp.plot(cmap='YlGnBu')
plt.show()
print()
print('accuracy: ', accuracy_score(y_test, y_pred))
nbtest.assert_allclose(accuracy_score(y_test, y_pred), 0.7966666666666666, atol=0.2599145158615723, test_id='34')

In [ ]:
import matplotlib.pyplot as plt
y_proba = dtc.predict_proba(X_test)
(fpr, tpr, thresholds) = roc_curve(y_test, y_proba[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)
nbtest.assert_allclose(roc_auc, 0.7767401710692505, atol=0.31520372185936313, test_id='35')
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()
print(sklearn.metrics.roc_auc_score(y_test, y_pred))
nbtest.assert_allclose(sklearn.metrics.roc_auc_score(y_test, y_pred), 0.7767401710692505, atol=0.31520372185936313, test_id='36')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=35, criterion='entropy', random_state=24)
nbtest_tmpvar_9 = rfc.fit(X_train, y_train)
nbtest.assert_sklearn_model(nbtest_tmpvar_9, '/home/verve/ml_nb_testing/comp_analysis/nbtest-gen/example/op/sklearn_model_37.pkl', test_id='37')
y_pred = rfc.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=rfc.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rfc.classes_)
disp.plot(cmap='YlGnBu')
plt.show()
print()
print('accuracy: ', accuracy_score(y_test, y_pred))
nbtest.assert_allclose(accuracy_score(y_test, y_pred), 0.8233333333333335, atol=0.16055459438389716, test_id='38')

In [ ]:
import matplotlib.pyplot as plt
y_proba = rfc.predict_proba(X_test)
(fpr, tpr, thresholds) = roc_curve(y_test, y_proba[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)
nbtest.assert_allclose(roc_auc, 0.90482373467913, atol=0.11338564795799509, test_id='39')
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()
print(sklearn.metrics.roc_auc_score(y_test, y_pred))
nbtest.assert_allclose(sklearn.metrics.roc_auc_score(y_test, y_pred), 0.7813223693084752, atol=0.21843646804798797, test_id='40')